In [1]:
# import libraries
import pandas  as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,confusion_matrix,roc_auc_score,classification_report

In [2]:
df=pd.read_excel(r"C:\Users\eprmitr\Desktop\TIGER\Mortgage Insurance Cross Sell.xls")

#Converting columns into lower case
df.columns = [i.lower() for i in df.columns]
df.head()

,ref,credit_score,final_grade,term,net_advance,apr,loan_type,mosaic,mosaic_class,time_at_address,...,cifas_detected,time_since_most_recent_outstandi,insurance_description,ppi,code,prdt_desc,category,ppi_single,ppi_joint,ppi_lci
0,1,918,A,36,3000.0,14.4,UnSecured,46,8,132,...,N,16,NaN,0,NaN,NaN,NaN,0,0,0
1,3,903,A,120,21000.0,7.9,secured,16,3,288,...,N,99,Life & Critical Illn,1,748.0,LIFE & CRITICAL ILLNESS,LCI,0,0,1
2,4,1060,X,78,7200.0,7.9,secured,17,3,276,...,N,50,Joint,1,719.0,LASCI JOINT,Joint,0,1,0
3,6,839,B,60,8000.0,16.9,UnSecured,47,8,48,...,N,99,Life & Critical Illn,1,748.0,LIFE & CRITICAL ILLNESS,LCI,0,0,1
4,7,1057,X,60,7650.0,7.4,UnSecured,55,10,156,...,N,99,Single,1,718.0,LASCI,Single,1,0,0


In [3]:
dff = df[df['ppi']==1]
dff.category = [i.lower() for i in dff.category]
dff.head()

C:\Users\eprmitr\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,ref,credit_score,final_grade,term,net_advance,apr,loan_type,mosaic,mosaic_class,time_at_address,...,cifas_detected,time_since_most_recent_outstandi,insurance_description,ppi,code,prdt_desc,category,ppi_single,ppi_joint,ppi_lci
1,3,903,A,120,21000.0,7.9,secured,16,3,288,...,N,99,Life & Critical Illn,1,748.0,LIFE & CRITICAL ILLNESS,lci,0,0,1
2,4,1060,X,78,7200.0,7.9,secured,17,3,276,...,N,50,Joint,1,719.0,LASCI JOINT,joint,0,1,0
3,6,839,B,60,8000.0,16.9,UnSecured,47,8,48,...,N,99,Life & Critical Illn,1,748.0,LIFE & CRITICAL ILLNESS,lci,0,0,1
4,7,1057,X,60,7650.0,7.4,UnSecured,55,10,156,...,N,99,Single,1,718.0,LASCI,single,1,0,0
5,8,913,A,120,33000.0,9.4,secured,59,11,18,...,N,99,Single,1,715.0,LASU,single,1,0,0


In [4]:
df.shape, dff.shape

((16383, 59), (9461, 59))

In [5]:
#changing categorical value to numerical values
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

cat_col = ['loan_type', 'residential_status','final_grade','employment_status','perm_temp_empl_ind',\
           'full_part_time_empl_ind','cifas_detected','gender','marital_status','bankruptcy_detected__sp_',\
           'telephone_indicator','current_account','access_card','visa_card','american_express',\
           'diners_card','cheque_guarantee','other_credit_store_card','payment_method']



#data = pd.get_dummies(dff, columns=cat_col)
data = le.fit_transform(data[cat_col])

In [6]:
y = dff.category
X = dff.drop(['insurance_description','prdt_desc','code','category','ppi_single','ppi_joint','ppi_lci','ppi'],axis=1) 
X = X.apply(le.fit_transform)
#X = X.values.reshape(-1,1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

In [8]:
# training a KNN classifier 
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 8)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

In [9]:
# accuracy on X_test 
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print (accuracy_knn)

# creating a confusion matrix 
cm_knn = confusion_matrix(y_test, y_pred_knn, labels=['single','joint','lci'])
print(cm_knn)

#auc_knn = metrics.roc_auc_score(y_test, y_pred_knn)
#print (auc_knn)

0.6136939983093829
[[1375   65  136]
 [ 263   23   26]
 [ 409   15   54]]


In [10]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(criterion='gini',max_depth=5, random_state=42)
dtree.fit(X_train, y_train) 
y_pred_dtree = dtree.predict(X_test)

In [11]:
accuracy_dtree = accuracy_score(y_pred_dtree, y_test)
print (accuracy_dtree)
#auc_dtree = metrics.roc_auc_score(y_test, y_pred_dtree)
#print (auc_dtree)
# creating a confusion matrix 
cm_dtree = confusion_matrix(y_test, y_pred_dtree, labels=['single','joint','lci'])
cm_dtree

0.6906170752324599


array([[1534,    6,   36],
       [ 311,    1,    0],
       [ 377,    2,   99]], dtype=int64)

In [12]:
i = ['KNN Classifier','DecisionTree Classifier']
d = {"Accuracy":[accuracy_knn,accuracy_dtree],"Confusion Matrix":[cm_knn,cm_dtree]}
result=pd.DataFrame(data=d,index=i)
result

,Accuracy,Confusion Matrix
KNN Classifier,0.613694,"[[1375, 65, 136], [263, 23, 26], [409, 15, 54]]"
DecisionTree Classifier,0.690617,"[[1534, 6, 36], [311, 1, 0], [377, 2, 99]]"


In [13]:
y_pred_dtree  = pd.DataFrame(y_pred_dtree)

df_out = X_test.reset_index()
#df_out["Actual"] = y_test.reset_index()["Columns_Name"]

In [14]:
y_test.shape,y_pred_dtree.shape,df_out.shape

((2366,), (2366, 1), (2366, 52))

In [15]:
y_test.value_counts()

single    1576
lci        478
joint      312
Name: category, dtype: int64

In [16]:
df_out["Actual"] = y_test.reset_index()["category"]
df_out["Prediction"] = y_pred_dtree
df_out[['credit_score','apr','loan_type','mosaic','mosaic_class','Actual','Prediction']].head()

,credit_score,apr,loan_type,mosaic,mosaic_class,Actual,Prediction
0,193,79,1,26,3,single,single
1,296,129,0,42,6,joint,single
2,242,105,0,44,7,single,single
3,123,197,0,10,1,single,single
4,401,79,0,24,3,single,single
